# Statistics with Julia from the ground up
#### A [JuliaCon 2021](https://juliacon.org/2021/) workshop by [Yoni Nazarathy](https://yoninazarathy.com/)

Many of the code examples for this workshop are adapted from [Statistics with Julia:
Fundamentals for Data Science, Machine Learning and Artificial Intelligence by Yoni Nazarathy and Hayden Klok](https://statisticswithjulia.org/).  

Also related (and recommended in this JuliaCon): 
* [Dataframes tutorial](https://github.com/bkamins/JuliaCon2021-DataFrames-Tutorial) or [here](https://pretalx.com/juliacon2021/talk/FXZXMB/) by Bogumił Kamiński.
* [Introduction to Bayesian Data Analysis](https://pretalx.com/juliacon2021/talk/J7BFBM/) by Kusti Skytén.
* Dozens of other very exciting talks...

<a id='home'></a>
# Table of Contents

1. [Why Julia?](#why-julia)
1. [What do you `mean`?](#what-do-you-mean)
1. [Something `rand`.](#something-rand)
1. [Do you still miss R? So Just `RCall`.](#just-rcall)
1. [Some `Plots`.](#some-plots)
1. [Your favorite `Distribution`.](#favorite-distribution)
1. [We love `DataFrames`.](#love-dataframes)
1. [Gotta have some basic inference.](#inference)
1. [Linear models at our core.](#linear-models)
1. [Basic Machine learning.](#basic-ml)


---
## Before we start

The tutorial was developed and tested under Julia 1.6.0. It is best to run it with the `Project.toml` and `Manifest.toml` files present in the working directory of the notebook. It also uses the following data files: 

In [ ]:
readdir("./data")

You will find all these files and this notebook in the [Github repo for this workshop](https://github.com/yoninazarathy/JuliaCon2021-StatisticsWithJuliaFromTheGroundUp). You can either "clone" the repo or download a zip file.

We use a [Jupyter Notebook](https://jupyter.org/). Here is a [quick reference sheet](edureka.co/blog/wp-content/uploads/2018/10/Jupyter_Notebook_CheatSheet_Edureka.pdf). Many other reserouces on the web as well for Jupyter - many of which use Python and not Julia, but Jupyter is the same. BTW the "J" in "Jupyter" is for "Julia".

Load the packages we will use:

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
Pkg.status()

In [ ]:
using Random, Statistics, LinearAlgebra, Dates #Shipped with Julia
using Distributions, StatsBase #Core statistics
using CSV, DataFrames #Basic Data
using Plots, StatsPlots, LaTeXStrings, Measures #Plotting and Output
using HypothesisTests, KernelDensity, GLM, Lasso, Clustering, MultivariateStats #Statistical/ML methods
using Flux, Metalhead #Deep learning 
using Combinatorics, SpecialFunctions, Roots #Mathematical misc.
using RDatasets, MLDatasets #Example datasets
#uncomment if using R:  using RCall #Interface with R

In [ ]:
# We run this before many examples for reproducibility
fix_seed!() = Random.seed!(0)

<a id='why-julia'></a>

---

# Why Julia?
[home](#home)

![Julia Curve](img/julia_curve.png)

### Some ways to run Julia

* REPL
    - As an application
    - Out of your shell 
    - As part of an IDE
* Jupyter (IJulia)
    - In your web browser
    - Jupyter Lab
* Google collab
* Pluto
* Visual Studio Code
* Legacy: Atom (Juno)
* JuliaHub
* In RMarkdown with IJulia (e.g. in R studio)
* ... 

### Key resources (My favorites)

* Main Julia Page: https://julialang.org/
* Docs: https://docs.julialang.org/
* Julia Express: https://github.com/bkamins/The-Julia-Express 
* Think Julia: https://www.oreilly.com/library/view/think-julia/9781492045021/
* MIT Course, computational thinking: https://computationalthinking.mit.edu/Spring21/ 
* A University of Queensland Course: https://courses.smp.uq.edu.au/MATH2504/
* Statistics with Julia: https://statisticswithjulia.org/ (use image gallary) 
* Package documentation: Searching for the package, e.g. `Plots.jl`, typically gets you to GitHub. From there find the docs.
* Julia Discourse: https://discourse.julialang.org/
* Julia Slack: https://julialang.org/slack/ 
* Your local Julia "club": E.g. in my area: https://www.meetup.com/en-AU/brisbane-julia-language-meetup/ 
* YouTube...


<a id='what-do-you-mean'></a>

---

# What do you `mean`?
[home](#home)

In [ ]:
fix_seed!()
data = rand(Normal(),5)

In [ ]:
n = length(data)

In [ ]:
sum(data)/n

In [ ]:
+(data...)/n

In [ ]:
? +

In [ ]:
"""
This is a function that takes in data and returns its sum.
"""
function my_sum(data)
    s = 0.0
    for d in data
        s += d
    end
    return s
end

In [ ]:
? my_sum

In [ ]:
my_sum(data)/length(data)

In [ ]:
mean(data)

In [ ]:
data'*ones(n)/n

In [ ]:
dot(data,ones(n))/n

In [ ]:
? dot

#### Doing it a little differently with the "running mean" formula

$$
\overline{X}_i = \frac{1}{i} X_i + \frac{i-1}{i} \overline{X}_{i-1}.
$$

In [ ]:
mn = 0
for i in 1:length(data)
    global mn = (1/i)*data[i] + (i-1)/i*mn
end
mn

In [ ]:
mn = 0
for (i,d) in enumerate(data)
    global mn = (1/i)*d + (i-1)/i*mn #Note that in Jupyter `global` isn't needed here but in the REPL it is.
end
mn

In [ ]:
function my_mean(data)
    mn = 0
    for (i,d) in enumerate(data)
        mn = (1/i)*d + (i-1)/i*mn
    end
    return mn
end

In [ ]:
my_mean(data)

In [ ]:
fix_seed!()
data = rand(Normal(),n) + im*rand(Normal(),n)

In [ ]:
mean(data)

In [ ]:
methods(mean)

In [ ]:
@which mean(data)

In [ ]:
methods(my_mean)

In [ ]:
function my_mean(data::Vector{Float64})::Float64
    mn = 0.0
    for (i,d) in enumerate(data)
        mn = (1/i)*d + (i-1)/i*mn
    end
    return mn
end

In [ ]:
methods(my_mean)

In [ ]:
@which my_mean(data)

In [ ]:
@which my_mean([1,2])

In [ ]:
@which my_mean([1.0,2.0])

In [ ]:
@which my_mean([1.0,2])

In [ ]:
function my_mean(data::Vector{T})::T where T
    mn = zero(data[begin])
    for (i,d) in enumerate(data)
        mn = (1/i)*d + (i-1)/i*mn
    end
    return mn
end

In [ ]:
methods(my_mean)

#### The mean of other types

In [ ]:
fix_seed!()
data = [rand(Normal(),2,2) for _ in 1:5]

In [ ]:
mean(data)

In [ ]:
my_mean(data)

In [ ]:
@which my_mean(data)

#### A glimpse under the hood

In [ ]:
@code_lowered my_mean(data)

In [ ]:
@code_llvm my_mean(data)

In [ ]:
@code_native my_mean(data)

#### An array of arrays

In [ ]:
fix_seed!()
data = [rand(Normal(μ,1),5) for μ in 1:5] #\mu + [TAB]

In [ ]:
mean.(data)

#### A matrix of data

In [ ]:
data = hcat(data...)

In [ ]:
heatmap(data',yflip=true)

In [ ]:
mean.(data) #does nothing useful

In [ ]:
mean(data,dims=1) #better!

#### Dictionaries

In [ ]:
data = Dict()
data[:cats] = rand(Normal(2.3,1),5)
data["dogs"] = rand(Normal(5.2,1),5)
data[25] = rand(Normal(7.8,1),5)
data

In [ ]:
keys(data)

In [ ]:
mean(data[:cats])

In [ ]:
mean(data["dogs"])

In [ ]:
mean(data["cats"])

In [ ]:
mean(data[Symbol("cats")])

In [ ]:
data[Symbol("cats")] |> mean

#### Tuples

In [ ]:
fix_seed!()
rn() = rand(Normal())
data = (rn(), rn(),rn())

In [ ]:
typeof(data)

In [ ]:
mean(data)

In [ ]:
data[2]

In [ ]:
data[2] = 7.8 #Tuples are immutable! 

#### Structs

In [ ]:
struct Person
    weight::Float64
    height::Float64
end

In [ ]:
someone = Person(102, 180)

In [ ]:
someone.weight = 103 #Structs are immutable unless you state immutable

In [ ]:
me = Person(102, 180)
you = Person(83, 170)
me + you #can't add persons just like that

In [ ]:
import Base: + 

function +(x::Person, y::Person)
    Person(x.weight + y.weight, x.height + y.height)
end

In [ ]:
me + you #Now you can

In [ ]:
import Base: /
function /(x::Person, n::Real)
    Person(x.weight/n, x.height/n)
end

In [ ]:
mean([me,you]) #With + and / (by number) we can compute the mean of persons

#### Dataframes 

In [ ]:
df = CSV.File("./data/temperatures.csv") |> DataFrame

In [ ]:
data = df.GoldCoast
mean(data)

#### All kind of other descriptive statistics

In [ ]:
println("Sample Mean: ", mean(data))
println("Harmonic ≤ Geometric ≤ Arithmetic ",      #\le + [TAB]
            (harmmean(data), geomean(data), mean(data)))

println("Sample Variance: ",var(data))
println("Sample Standard Deviation: ",std(data))
println("Minimum: ", minimum(data))
println("Maximum: ", maximum(data))
println("Median: ", median(data))
println("95th percentile: ", percentile(data, 95))
println("0.95 quantile: ", quantile(data, 0.95))
println("Interquartile range: ", iqr(data),"\n")

summarystats(data)

In [ ]:
min(5,2,-3) #Minimum of individual function arguments

In [ ]:
minimum([5,2,-3]) #Minimum of an array (AbstractArray)

In [ ]:
min([5,2,-3]) #doesn't work

In [ ]:
min([5,2,-3]...) #The splat operator passes the arguments 

In [ ]:
fix_seed!()
data = rand(Normal(),10^6);

In [ ]:
@time begin
    minimum(data)
end

In [ ]:
@time begin
    min(data...)
end

<a id='something-rand'></a>
# Something `rand`.
[home](#home)

#### The Monty Hall Problem 

In [ ]:
fix_seed!()

function montyHall(switch_policy)
    prize, choice = rand(1:3), rand(1:3)
    if prize == choice
        revealed = rand(setdiff(1:3,choice))
    else
        revealed = rand(setdiff(1:3,[prize,choice]))
    end

    if switch_policy
        choice = setdiff(1:3,[revealed,choice])[1]
    end
    return choice == prize
end

N = 10^6
println("Success probability with policy I (stay): ",  sum([montyHall(false) for _ in 1:N])/N)
println("Success probability with policy II (switch): ", sum([montyHall(true) for _ in 1:N])/N)

#### Common random numbers

$$
X \sim \text { Uniform }(0,2 \lambda(1-\lambda))
$$

$$
\mathbb{E}_{\lambda}[X]=\lambda(1-\lambda)
$$

In [ ]:
seed = 1
N = 100
λ_grid = 0.01:0.01:0.99

theorM(λ) = mean(Uniform(0,2λ*(1-λ)))
estM(λ) = mean(rand(Uniform(0,2λ*(1-λ)),N))

function estM(λ, seed)
    Random.seed!(seed)
    estM(λ)
end

trueM = theorM.(λ_grid)
estM0 = estM.(λ_grid)
estMCRN = estM.(λ_grid,seed)

plot(λ_grid, trueM, c=:black, label="Expected curve")
plot!(λ_grid, estM0, c=:blue, label="No CRN estiamte")
plot!(λ_grid, estMCRN, c=:red, label="CRN estimate", 
    xlims=(0,1), ylims=(0,0.4), xlabel=L"\lambda", ylabel = "Mean")

#### Multiple RNGs

Say we want to estimate the mean of 
$$
X=\sum_{i=1}^{N} Z_{i}
$$
with $N \sim \operatorname{Poisson}(K \lambda)$ and $Z_{i} \sim \text { Uniform }(0,2(1-\lambda))$  with  $\lambda \in(0,1)$  and $K>0$.

$$
\mathbb{E}_{\lambda}[X]=K \lambda(1-\lambda)
$$

In [ ]:
K = 50

prn(λ,rng) = quantile(Poisson(λ),rand(rng))
zDist(λ) = Uniform(0,2*(1-λ))

rv(λ,rng) = sum([rand(rng,zDist(λ)) for _ in 1:prn(K*λ, rng)])
rv2(λ,rng1,rng2) = sum([rand(rng1,zDist(λ)) for _ in 1:prn(K*λ,rng2)])

mEst(λ,rng) = mean([rv(λ,rng) for _ in 1:N])
mEst2(λ,rng1,rng2) = mean([rv2(λ,rng1,rng2) for _ in 1:N])

#No Common Random Numbers (CRN)
function mGraph0(seed)
    singleRng = MersenneTwister(seed)
    [mEst(λ,singleRng) for λ in λ_grid]
end

#CRN but single random number generator (RNG)
mGraph1(seed) = [mEst(λ,MersenneTwister(seed)) for λ in λ_grid]

#CRN with two random number generators
mGraph2(seed1,seed2) = [mEst2(λ,MersenneTwister(seed1), MersenneTwister(seed2)) for λ in λ_grid]

plot(λ_grid,mGraph0(1987), c=:red, label="No CRN")
plot!(λ_grid,mGraph1(1987), c=:green, label="CRN and one RNG")
plot!(λ_grid,mGraph2(1987,1988), c=:blue, label="CRN and two RNG's", xlims=(0,1),ylims=(0,14),
    xlabel=L"\lambda", ylabel = "Mean")

In [ ]:
argMaxλ(graph) = λ_grid[findmax(graph)[2]]

M = 10^3
std0 = std([argMaxλ(mGraph0(seed)) for seed in 1:M])
std1 = std([argMaxλ(mGraph1(seed)) for seed in 1:M])
std2 = std([argMaxλ(mGraph2(seed,seed+M)) for seed in 1:M])

println("Standard deviation with no CRN: ", std0)
println("Standard deviation with CRN and single RNG: ", std1)
println("Standard deviation with CRN and two RNGs: ", std2)

<a id='just-rcall'></a>
# Do you still miss R? So Just `RCall`.
[home](#home)

See the docs for [RCall.jl](https://juliainterop.github.io/RCall.jl/stable/)

In [ ]:
using RCall

data1 = DataFrame(CSV.File("./data/machine1.csv", header=false))[:,1]
data2 = DataFrame(CSV.File("./data/machine2.csv", header=false))[:,1]
data3 = DataFrame(CSV.File("./data/machine3.csv", header=false))[:,1]

function R_ANOVA(allData)
    data = vcat([ [x fill(i, length(x))] for (i, x) in enumerate(allData) ]...)
    df = DataFrame(data, [:Diameter, :MachNo])
    @rput df

    R"""
    df$MachNo <- as.factor(df$MachNo)
    anova <- summary(aov( Diameter ~ MachNo, data=df))
    fVal <- anova[[1]]["F value"][[1]][1]
    pVal <- anova[[1]]["Pr(>F)"][[1]][1]
    """
    println("R ANOVA f-value: ", @rget fVal)
    println("R ANOVA p-value: ", @rget pVal)
end

R_ANOVA([data1, data2, data3])

To use Julia from R, use [JuliaCall](https://cran.r-project.org/web/packages/JuliaCall/index.html)

<a id='some-plots'></a>
# Some `Plots`.
[home](#home)

See example [image gallery](https://statisticswithjulia.org/gallery.html) with code - part of "Statistics with Julia" book.

See docs for [Plots.jl](http://docs.juliaplots.org/latest//) and [StatsPlots.jl](https://github.com/JuliaPlots/StatsPlots.jl)

In [ ]:
function hailLength(x::Int)
    n = 0
    while x != 1
        if x % 2 == 0
            x = x ÷ 2 #\div + [TAB]
        else
            x = 3x +1
        end
        n += 1
    end
    return n
end

lengths = [hailLength(x₀) for x₀ in 2:10^7] #\_0 + [TAB]

histogram(lengths, bins=1000, normed=:true, 
    fill=(:blue, true), la=0, legend=:none,
    xlims=(0, 500), ylims=(0, 0.012),
    xlabel="Length", ylabel="Frequency")

In [ ]:
data = CSV.File("./data/temperatures.csv") |> DataFrame
brisbane = data.Brisbane
goldcoast = data.GoldCoast

diff = brisbane - goldcoast
dates = [Date(
            Year(data.Year[i]), 
            Month(data.Month[i]), 
            Day(data.Day[i])
        ) for i in 1:nrow(data)]

fortnightRange = 250:263
brisFortnight = brisbane[fortnightRange]
goldFortnight = goldcoast[fortnightRange]

p1 = plot(dates, [brisbane goldcoast], 
        c=[:blue :red], xlabel="Time", ylabel="Temperature", label=["Brisbane" "Gold Coast"])
p2 = plot(dates[fortnightRange], [brisFortnight goldFortnight], xticks=:none,
        c=[:blue :red], m=(:dot, 5, Plots.stroke(1)), ylabel="Temperature",
        label=["Brisbane" "Gold Coast"], xlabel="Time within one fortnight")
p3 = plot(dates, diff, 
        c=:black, ylabel="Temperature Difference",legend=false)
p4 = histogram(diff, bins=-4:0.5:6, 
        ylims=(0,140), legend = false,
        xlabel="Temperature Difference", ylabel="Frequency")
plot(p1,p2,p3,p4, size = (800,500), margin = 5mm)

In [ ]:
fix_seed!()

μ₁, σ₁ = 10, 5 #\mu + [TAB] \_1 + [TAB] etc...
μ₂, σ₂ = 40, 12
dist1, dist2 = Normal(μ₁,σ₁), Normal(μ₂,σ₂)
p = 0.3
mixRv() = (rand() <= p) ? rand(dist1) : rand(dist2)

n = 2000
data = [mixRv() for _ in 1:n]

density(data, c=:blue, label="Density via StatsPlots", xlims=(-20,80), ylims=(0,0.035))
stephist!(data, bins=50, c=:black, norm=true, label="Histogram", xlabel="x", ylabel = "Density")

In [ ]:
mixPDF(x) = p*pdf(dist1,x) + (1-p)*pdf(dist2,x)

kdeDist = kde(data)

xGrid = -20:0.1:80
pdfKDE = pdf(kdeDist,xGrid)

p1 = plot(xGrid, pdfKDE, c=:blue, label="KDE PDF")
stephist!(data, bins=50, c=:black, normed=:true, label="Histogram")
plot!(xGrid, mixPDF.(xGrid), c=:red, label="Underlying PDF",
    xlims=(-20,80), ylims=(0,0.035), legend=:topleft,
    xlabel="X", ylabel = "Density")

hVals = [0.5,2,10]
kdeS = [kde(data,bandwidth=h) for h in hVals]

p2 = plot(xGrid, pdf(kdeS[1],xGrid), c = :green, label= "h=$(hVals[1])")
plot!(xGrid, pdf(kdeS[2],xGrid), c = :blue, label= "h=$(hVals[2])")
plot!(xGrid, pdf(kdeS[3],xGrid), c = :purple, label= "h=$(hVals[3])",
    xlims=(-20,80), ylims=(0,0.035), legend=:topleft, 
    xlabel="X", ylabel = "Density")

plot(p1,p2,size = (800,400))

In [ ]:
fix_seed!()

mixCDF(x) = p*cdf(dist1,x) + (1-p)*cdf(dist2,x)

n = [30, 100]
data1 = [mixRv() for _ in 1:n[1]]
data2 = [mixRv() for _ in 1:n[2]]

empiricalCDF1 = ecdf(data1)
empiricalCDF2 = ecdf(data2)

xGrid = -10:0.1:80
plot(xGrid,empiricalCDF1.(xGrid), c=:blue, label="ECDF with n = $(n[1])")
plot!(xGrid,empiricalCDF2.(xGrid), c=:red, label="ECDF with n = $(n[2])")
plot!(xGrid, mixCDF.(xGrid), c=:black, label="Underlying CDF",
    xlims=(-10,80), ylims=(0,1), 
    xlabel="x", ylabel="Probability", legend=:topleft)

In [ ]:
fix_seed!()

b1, b2 = 0.5 , 2
dist1, dist2, = Beta(b1,b1), Beta(b2,b2)
 
n = 2000
data1 = rand(dist1,n)
data2 = rand(dist2,n)

stephist(data1, bins=15, label = "beta($b1,$b1)", c = :red, normed = true)
p1 = stephist!(data2, bins=15, label = "beta($b2,$b2)",
        c = :blue, xlabel="x", ylabel="Density",normed = true)

p2 = qqplot(data1, data2, c=:black, ms=1, msw =0,
        xlabel="Quantiles for beta($b1,$b1) sample",
        ylabel="Quantiles for beta($b2,$b2) sample",
        legend=false)

plot(p1, p2, size=(800,400), margin = 5mm)

In [ ]:
data1 = parse.(Float64, readlines("./data/machine1.csv"))
data2 = parse.(Float64, readlines("./data/machine2.csv"))
data3 = parse.(Float64, readlines("./data/machine3.csv"))

boxplot([data1,data2,data3], c=[:blue :red :green], label="", 
    xticks=([1:1:3;], ["1", "2", "3"]), xlabel="Machine type",
    ylabel="Pipe Diameter (mm)")

<a id='favorite-distribution'></a>
# Your favorite `Distribution`.
[home](#home)

See [Docs for `Distributions.jl`](https://juliastats.org/Distributions.jl/latest/)

In [ ]:
dist = Normal(2.5,1.2)

In [ ]:
mean(dist), var(dist)

In [ ]:
fix_seed!()
rand(dist,3,2,5) #a 3-tensor

In [ ]:
? NegativeBinomial

In [ ]:
fix_seed!()

function rouletteSpins(r,p)
    x = 0
    wins = 0
    while true
        x += 1
        if rand() < p
            wins += 1
            if wins == r
                return x
            end
        end
    end
end

r, p, N = 5, 18/37,10^6
xGrid = r:r+15

mcEstimate = counts([rouletteSpins(r,p) for _ in 1:N],xGrid)/N

nbDist = NegativeBinomial(r,p)
nbPmf = [pdf(nbDist,x-r) for x in xGrid]

plot( xGrid, mcEstimate, 
    line=:stem, marker=:circle, c=:blue, 
    ms=10, msw=0, lw=4, label="MC estimate")
plot!( xGrid, nbPmf, line=:stem, 
    marker=:xcross, c=:red, ms=6, msw=0, lw=2, label="PMF", 
    xlims=(0,maximum(xGrid)), ylims=(0,0.2), 
    xlabel="x", ylabel="Probability")

In [ ]:
function plot_it(N)
    fix_seed!()
    mcEstimate = counts([rouletteSpins(r,p) for _ in 1:N],xGrid)/N

    nbDist = NegativeBinomial(r,p)
    nbPmf = [pdf(nbDist,x-r) for x in xGrid]

    plot( xGrid, mcEstimate, 
        line=:stem, marker=:circle, c=:blue, 
        ms=10, msw=0, lw=4, label="MC estimate")
    plot!( xGrid, nbPmf, line=:stem, 
        marker=:xcross, c=:red, ms=6, msw=0, lw=2, label="PMF", 
        xlims=(0,maximum(xGrid)), ylims=(0,0.2), 
        xlabel="x", ylabel="Probability",
        title="N = $N")
end

anim = Animation()

for N in union(10:10:100,200:100,1000,2000:1000:10^4)
    plot_it(N)
    frame(anim)
end

gif(anim,"sample_animation.gif",fps = 3)

In [ ]:
plot((x)->pdf(Normal(),x),label = "Standard Normal")
plot!((x)->pdf(Cauchy(),x),label = "Standard Cauchy")

In [ ]:
fix_seed!()

n = 10^6
data = rand(Normal(), n)
#data = rand(Cauchy(),n) #Try this instead
averages = accumulate(+,data) ./ (1:n)

plot( 1:n, averages, 
    c=:blue, legend=:none, xscale=:log10, xlims=(1,n), xlabel="n", ylabel="Running average")

In [ ]:
fix_seed!()

N = 10^4

Σ = [ 6 4 ;
         4 9]
μ = [15 ; 
       20]
A = cholesky(Σ).L

rngGens = [()->rand(Normal()), 
           ()->rand(Uniform(-sqrt(3),sqrt(3))),
           ()->rand(Exponential())-1]

rv(rg) = A*[rg(),rg()] + μ
    
data = [[rv(r) for _ in 1:N] for r in rngGens]

stats(data) = begin
    data1, data2 = first.(data),last.(data)
    println(round(mean(data1),digits=2), "\t",round(mean(data2),digits=2),"\t",
            round(var(data1),digits=2), "\t", round(var(data2),digits=2), "\t",
            round(cov(data1,data2),digits=2))
end

println("Mean1\tMean2\tVar1\tVar2\tCov")
for d in data
    stats(d)
end

scatter(first.(data[1]), last.(data[1]), c=:blue, ms=1, msw=0, label="Normal")
scatter!(first.(data[2]), last.(data[2]), c=:red, ms=1, msw=0, label="Uniform")
scatter!(first.(data[3]), last.(data[3]), c=:green, ms=1, msw=0,label="Exponential",
    xlims=(0,40), ylims=(0,40), legend=:bottomright, ratio=:equal,
    xlabel=L"X_1", ylabel=L"X_2")

In [ ]:
dist = MvNormal([1,1],[2 2.3; 2.3 4])

In [ ]:
cor(dist)

In [ ]:
rand(dist,20)

See also [TruncatedDistributions.jl](https://github.com/yoninazarathy/TruncatedDistributions.jl) - still in progress as of July 2021.

<a id='love-dataframes'></a>
# We love `DataFrames`.
[home](#home)

See also:

* Yesterday's (recorded) [Dataframes tutorial](https://github.com/bkamins/JuliaCon2021-DataFrames-Tutorial) or [here](https://pretalx.com/juliacon2021/talk/FXZXMB/) by Bogumił Kamiński.
* Docs for [DataFrames.jl](https://dataframes.juliadata.org/stable/).

In [ ]:
ENV["LINES"] = 10
data = CSV.File("./data/purchaseData.csv") |> DataFrame

In [ ]:
size(data)

In [ ]:
names(data)

In [ ]:
first(data, 6)

In [ ]:
describe(data)

In [ ]:
println("Grade of person 1: ", data[1, 3], 
        ", ", data[1,:Grade], 
        ", ", data.Grade[1])

In [ ]:
data[[1,2,4], :]

In [ ]:
data[13:15, :Name]

In [ ]:
data.Name[13:15]

In [ ]:
data[13:15, [:Name]]

#### Mising values

In [ ]:
mean(data.Price)

In [ ]:
ismissing.(data.Price) |> sum

In [ ]:
mean(skipmissing(data.Price))

In [ ]:
data.Grade[1:4]

In [ ]:
coalesce.(data.Grade, "QQ")[1:4]

In [ ]:
dropmissing(data,:Price)

In [ ]:
completecases(data) |> sum

In [ ]:
full_obs = completecases(data) |> findall

In [ ]:
data[setdiff(1:size(data,1), full_obs),:]

<a id='inference'></a>
# Gotta have some basic inference.
[home](#home)

#### Point estimation

In [ ]:
fix_seed!()

N = 10^5
nMin, nStep, nMax = 10, 10, 200
nn = Int(nMax/nStep)
sampleSizes = nMin:nStep:nMax
trueB = 5
trueDist = Uniform(-2, trueB)

#MLE for the upper bound
MLEest(data) = maximum(data)

#Method of moments estimator for the upper bound
MMest(data)  = mean(data) + sqrt(3)*std(data)

res = Dict{Symbol,Array{Float64}}( ((sym) -> sym => Array{Float64}(undef,nn)).(
                                [:MSeMLE,:MSeMM, :VarMLE,:VarMM,:BiasMLE,:BiasMM]))

for (i, n) in enumerate(sampleSizes)
    mleEst, mmEst = Array{Float64}(undef, N), Array{Float64}(undef, N) 
    for j in 1:N
        sample    = rand(trueDist,n)
        mleEst[j] = MLEest(sample)
        mmEst[j]  = MMest(sample)
    end
    meanMLE, meanMM = mean(mleEst), mean(mmEst)
    varMLE, varMM = var(mleEst), var(mmEst)

    res[:MSeMLE][i] = varMLE + (meanMLE - trueB)^2
    res[:MSeMM][i] = varMM + (meanMM - trueB)^2
    res[:VarMLE][i] = varMLE
    res[:VarMM][i] = varMM
    res[:BiasMLE][i] = meanMLE - trueB
    res[:BiasMM][i] = meanMM - trueB
end

p1 = scatter(sampleSizes, [res[:MSeMLE] res[:MSeMM]], c=[:blue :red],
    label=["Mean sq.err (MLE)" "Mean sq.err (MM)"])
p2 = scatter(sampleSizes, [res[:VarMLE] res[:VarMM]], c=[:blue :red],
    label=["Variance (MLE)" "Variance (MM)"])
p3 = scatter(sampleSizes, [res[:BiasMLE] res[:BiasMM]], c=[:blue :red],
    label=["Bias (MLE)" "Bias (MM)"],legend = :bottomright)

plot(p1, p2, p3, ms=2, shape=:circle, xlabel="n", 
    layout=(1,3), size=(800, 300), margin = 2mm)

In [ ]:
fix_seed!()

eq(alpha, xb, xbl) = log(alpha) - digamma(alpha) - log(xb) + xbl

actualAlpha, actualLambda = 2, 3
gammaDist = Gamma(actualAlpha,1/actualLambda)

function mle(sample)
    alpha  = find_zero( (a)->eq(a,mean(sample),mean(log.(sample))), 1)
    lambda = alpha/mean(sample)
    return [alpha,lambda]
end

N = 10^3

mles10   = [mle(rand(gammaDist,10)) for _ in 1:N]
mles100  = [mle(rand(gammaDist,100)) for _ in 1:N]
mles1000 = [mle(rand(gammaDist,1000)) for _ in 1:N]

scatter(first.(mles10), last.(mles10), 
    c=:blue, ms=1, msw=0, label="n = 10")
scatter!(first.(mles100), last.(mles100), 
    c=:red, ms=1, msw=0, label="n = 100")
scatter!(first.(mles1000), last.(mles1000), 
    c=:green, ms=1, msw=0, label="n = 1000", 
    xlims=(0,6), ylims=(0,8), xlabel=L"\alpha", ylabel=L"\lambda")

#### Confidence Intervals

In [ ]:
data1 = (CSV.File("./data/machine1.csv", header=false) |> DataFrame)[:,1]
data2 = (CSV.File("./data/machine2.csv", header=false) |> DataFrame)[:,1]
boxplot([data1,data2], c=[:blue :red], label="", 
    xticks=([1:2;], ["1", "2"]), xlabel="Machine type",
    ylabel="Pipe Diameter (mm)")

In [ ]:
α = 0.05 #\alpha + [TAB]
confint(EqualVarianceTTest(data1,data2),α)

In [ ]:
#Doing it manually
xBar1, xBar2 = mean(data1), mean(data2)
n1, n2 = length(data1), length(data2)
t = quantile(TDist(n1+n2-2),1-α/2)

s1, s2 = std(data1), std(data2)
sP = sqrt(((n1-1)*s1^2 + (n2-1)*s2^2) / (n1+n2-2))

(xBar1 - xBar2 - t*sP* sqrt(1/n1 + 1/n2), xBar1 - xBar2 + t*sP* sqrt(1/n1 + 1/n2))

#### Hypothesis Tests

In [ ]:
δ₀ = 0
test_result =UnequalVarianceTTest(data1, data2, δ₀)

In [ ]:
test_result.df

In [ ]:
pvalue(test_result)

In [ ]:
methods(pvalue)

In [ ]:
@which pvalue(test_result)

In [ ]:
xBar1, s1, n1 = mean(data1), std(data1), length(data1)
xBar2, s2, n2 = mean(data2), std(data2), length(data2)

v = ( s1^2/n1 + s2^2/n2 )^2 / ( (s1^2/n1)^2/(n1-1) + (s2^2/n2)^2/(n2-1)  )
testStatistic = ( xBar1-xBar2 - δ₀ )  / sqrt( s1^2/n1 + s2^2/n2 )
pVal = 2*ccdf(TDist(v), abs(testStatistic))

println("Manually calculated degrees of freedom, v: ", v)
println("Manually calculated test statistic: ", testStatistic)
println("Manually calculated p-value: ", pVal, "\n")

In [ ]:
data = CSV.File("./data/fertilizer.csv") |> DataFrame

control = data.Control
fertilizer = data.FertilizerX

subGroups = collect(combinations([control;fertilizer],10))

meanFert = mean(fertilizer)
pVal = sum([mean(i) >= meanFert for i in subGroups])/length(subGroups)
println("p-value = ", pVal)

<a id='linear-models'></a>
# Linear models at our core.
[home](#home)

In [ ]:
data = CSV.File("./data/L1L2data.csv") |> DataFrame
xVals, yVals = data[:,1], data[:,2]
scatter(xVals,yVals,label=false,xlim=(0,10),ylim=(0,10),xlabel="X", ylabel="Y")

In [ ]:
n = length(xVals)
A = [ones(n) xVals] #The "design matrix"

In [ ]:
# Approach A
xBar, yBar = mean(xVals),mean(yVals)
sXX, sXY = ones(n)'*(xVals.-xBar).^2 , dot(xVals.-xBar,yVals.-yBar)
b1A = sXY/sXX
b0A = yBar - b1A*xBar

# Approach B
b1B = cor(xVals,yVals)*(std(yVals)/std(xVals))
b0B = yBar - b1B*xBar

# Approach C
b0C, b1C = A'A \ A'yVals

# Approach D
Adag = inv(A'*A)*A'
b0D, b1D = Adag*yVals

# Approach E
b0E, b1E = pinv(A)*yVals

# Approach F
b0F, b1F = A\yVals

# Approach G
F = qr(A)
Q, R = F.Q, F.R
b0G, b1G = (inv(R)*Q')*yVals

# Approach H
F = svd(A)
V, Sp, Us = F.V, Diagonal(1 ./ F.S), F.U'
b0H, b1H = (V*Sp*Us)*yVals

# Approach I
eta, eps = 0.002, 10^-6.
b, bPrev = [0,0], [1,1]
while norm(bPrev-b) >= eps
    global bPrev = b
    global b = b - eta*2*A'*(A*b - yVals)
end
b0I, b1I = b[1], b[2]

# Approach J
modelJ = lm(@formula(Y ~ X), data)
b0J, b1J = coef(modelJ)

# Approach K
modelK = glm(@formula(Y ~ X), data, Normal())
b0K, b1K = coef(modelK)

println(round.([b0A,b0B,b0C,b0D,b0E,b0F,b0G,b0H,b0I,b0J,b0K],digits=3))
println(round.([b1A,b1B,b1C,b1D,b1E,b1F,b1G,b1H,b1I,b1J,b1K],digits=3))

In [ ]:
data = CSV.File("./data/weightHeight.csv") |> DataFrame

lm1 = lm(@formula(Height ~ Weight), data)
lm2 = fit(LinearModel,@formula(Height ~ Weight), data)

glm1 = glm(@formula(Height ~ Weight), data, Normal(), IdentityLink())
glm2 = fit(GeneralizedLinearModel,@formula(Height ~ Weight), data, Normal(), IdentityLink())

println("***Output of LM Model:")
println(lm1)
println("\n***Output of GLM Model:")
println(glm1)

pred(x) = coef(lm1)'*[1, x]

println("\n***Individual methods applied to model output:")
println("Deviance: ",deviance(lm1))
println("Standard error: ",stderror(lm1))
println("Degrees of freedom: ",dof_residual(lm1))
println("Covariance matrix: ",vcov(lm1))

yVals = data.Height
SStotal = sum((yVals .- mean(yVals)).^2)

println("R squared (calculated in two ways):",r2(lm1),
        ",\t", 1 - deviance(lm1)/SStotal)

println("MSE (calculated in two ways: ",deviance(lm1)/dof_residual(lm1),
        ",\t",sum((pred.(data.Weight) - data.Height).^2)/(size(data)[1] - 2))

xlims = [minimum(data.Weight), maximum(data.Weight)]
scatter(data.Weight, data.Height, c=:blue, msw=0)
plot!(xlims, pred.(xlims), 
    c=:red, xlims=(xlims), 
    xlabel="Weight (kg)", ylabel="Height (cm)", legend=:none)

#### Multiple variables

In [ ]:
df = RDatasets.dataset("MASS", "cpus")
df.Freq = map( x->10^9/x , df.CycT)

model = lm(@formula(Perf ~ MMax + Cach + ChMax + Freq), df)
pred(x) = round(coef(model)'*vcat(1,x),digits = 3)

println("n = ", size(df)[1])
println("(Avg,Std) of observed performance: ", (mean(df.Perf),std(df.Perf)))
println(model)
println("Estimated performance for computer A: ", pred([32000, 32, 32, 4*10^7]))
println("Estimated performance for computer B: ", pred([32000, 16, 32, 6*10^7]))

#### Categorical variables

In [ ]:
df = CSV.File("./data/weightHeight.csv") |> DataFrame
n = size(df)[1]
df[shuffle(1:n),:] = df
df[[10,40,60,130,140,175,190,200],:Sex] .= "O1"
df[[9,44,63,132,138,172,192,199],:Sex] .= "O2"

model = lm(@formula(Height ~ Weight + Sex), df,
        contrasts=Dict(:Sex=>DummyCoding(base="F",levels=["M","O1","O2","F"])))
b0, b1, b2, b3, b4  = coef(model)
pred(weight,sex) = b0+b1*weight+b2*(sex=="M")+b3*(sex=="O1")+b3*(sex=="O2")
println(model)

males = df[df.Sex .== "M",:]
females = df[df.Sex .== "F",:]
other = df[(df.Sex .!= "M") .& (df.Sex .!= "F"),:]

xlim = [minimum(df.Weight), maximum(df.Weight)]
scatter(males.Weight, males.Height, c=:blue, msw=0, label="Males")
plot!(xlim, pred.(xlim,"M"), c=:blue, label="Male model")

scatter!(females.Weight, females.Height, c=:red, msw=0, label="Females")
plot!(xlim, pred.(xlim,"F"), 
    c=:red, label="Female model", xlims=(xlim), 
    xlabel="Weight (kg)", ylabel="Height (cm)", legend=:topleft)

scatter!(other.Weight, other.Height, c=:green, msw=0, label="Other")

#### Variable selection with LASSO

In [ ]:
df = RDatasets.dataset("MASS", "cpus")
df.Freq = map( x->10^9/x , df.CycT)
df = df[:, [:Perf, :Freq, :MMin, :MMax, :Cach, :ChMin, :ChMax]]
X = [df.Freq df.MMin df.MMax df.Cach df.ChMin df.ChMax]
Y = df.Perf

targetNumVars = 3

lambdaStep = 0.2
lamGrid = collect(0:lambdaStep:150)
lassoFit = fit(LassoPath,X, Y, λ = lamGrid);
dd = Array(lassoFit.coefs)'
nV = sum(dd .!= 0.0 ,dims=2)

goodLambda = lamGrid[findfirst((n)->n==targetNumVars,nV)]
newFit = fit(LassoPath,X, Y, λ = [goodLambda - lambdaStep, goodLambda])
println(newFit)
println("Coefficients: ", Array(newFit.coefs)'[2,:])

p1 = plot(lassoFit.λ, dd, label = ["Freq" "MMin" "MMax" "Cach" "ChMin" "ChMax"],
    ylabel = "Coefficient Value")
plot!([goodLambda,goodLambda],[-1,1.5],c=:black, lw=2, label = "Model cut-off")

p2 = plot(lassoFit.λ,nV, ylabel = "Number of Variables",legend = false)
plot!([goodLambda,goodLambda],[0,6],c=:black, lw=2, label = "Model cut-off")

plot(p1,p2,xlabel= L"\lambda", margin = 5mm, size = (800,400))

#### Generalized linear models

In [ ]:
df = RDatasets.dataset("MASS", "cpus")
n = size(df)[1]
df = df[shuffle(1:n),:]

pTest = 0.2
lastTindex = Int(floor(n*(1-pTest)))
numTest = n - lastTindex

train = df[1:lastTindex,:]
test = df[lastTindex+1:n,:]

form = @formula(Perf~CycT+MMin+MMax+Cach+ChMin+ChMax)
model1 = glm(form, train, Normal(),  IdentityLink())
model2 = glm(form, train, Poisson(), LogLink())
model3 = glm(form, train, Gamma(),  InverseLink())

invIdenityLink(x) = x
invLogLink(x) = exp(x)
invInverseLink(x) = 1/x

A = [ones(numTest) test.CycT test.MMin test.MMax test.Cach test.ChMin test.ChMax]
pred1 = invIdenityLink.(A*coef(model1))
pred2 = invLogLink.(A*coef(model2))
pred3 = invInverseLink.(A*coef(model3))

actual = test.Perf
lossModel1 = norm(pred1 - actual)
lossModel2 = norm(pred2 - actual)
lossModel3 = norm(pred3 - actual)

println("Model 1: ", coef(model1))
println("Model 2: ", coef(model2))
println("Model 3: ", coef(model3))
println("\nLoss of models 1,2,3: ",(lossModel1 ,lossModel2, lossModel3))

<a id='basic-ml'></a>
# Basic Machine learning.
[home](#home)

Some resources:

* Docs for [MLJ.jl](https://alan-turing-institute.github.io/MLJ.jl/dev/) - not used here.
* Docs for [Scikitlearn.jl](https://github.com/cstjean/ScikitLearn.jl) - not used here.
* Docs for [Flux.jl](https://fluxml.ai/Flux.jl/stable/)
* [The Mathematical Engineering of Deep Learning](https://deeplearningmath.org/amsi-summer-school-course-2021.html)
* ...

#### Applying off the shelf deep neural networks (see https://github.com/FluxML/Metalhead.jl) 

In [ ]:
vgg = VGG19(); #A neural network model VGG19 (about 500Mb of parameters, from 2014)
vgg.layers

In [ ]:
#download an arbitrary image and try to classify it
download("https://deeplearningmath.org/data/images/appleFruit.jpg","appleFruit.jpg");
img = load("appleFruit.jpg")

In [ ]:
@time begin
    classify(vgg,img) #vgg is the model
end

#### Clustering with $k$-means

In [ ]:
fix_seed!()

K = 3
df = RDatasets.dataset("cluster", "xclara")
data = Matrix(df)'

seeds = initseeds(:rand, data, K)
xclaraKmeans = kmeans(data, K, init = seeds)

println("Number of clusters: ", nclusters(xclaraKmeans))
println("Counts of clusters: ", counts(xclaraKmeans))

df.Group  = assignments(xclaraKmeans)

p1 = scatter(df[:, :V1], df[:, :V2], c=:blue, msw=0)
     scatter!(df[seeds, :V1], df[seeds, :V2], markersize=12, c=:red, msw=0)

p2 = scatter( df[df.Group .== 1, :V1], df[df.Group .== 1, :V2], c=:blue, msw=0)
     scatter!( df[df.Group .== 2, :V1], df[df.Group .== 2, :V2], c=:red, msw=0)
     scatter!( df[df.Group .== 3, :V1], df[df.Group .== 3, :V2], c=:green, msw=0)

plot(p1,p2,legend=:none,ratio=:equal, size=(800,400), xlabel="V1", ylabel="V2", margin = 5mm)

#### The MNIST digits dataset

In [ ]:
using MLDatasets

In [ ]:
train_data = MLDatasets.MNIST.traindata(Float64)

imgs = train_data[1]
@show typeof(imgs)
@show size(imgs)

labels = train_data[2]
@show typeof(labels);

In [ ]:
test_data = MLDatasets.MNIST.testdata(Float64)
test_imgs = test_data[1]
test_labels = test_data[2]
@show size(test_imgs);

In [ ]:
n_train, n_test = length(labels), length(test_labels)

In [ ]:
[Gray.(train_data[1][:,:,k]') for k in 1:5]

In [ ]:
X = vcat([vec(imgs[:,:,k])' for k in 1:last(size(imgs))]...)
@show size(X)
heatmap(X,legend=false)

#### Principal component analysis (PCA)

In [ ]:
pca = fit(PCA, X'; maxoutdim=2)
M = projection(pca)

args = (ms=0.8, msw=0, xlims=(-5,12.5), ylims=(-7.5,7.5),
            legend = :topright, xlabel="PC 1", ylabel="PC 2")

function compareDigits(dA,dB)
    xA = X[labels .== dA, :]'
    xB = X[labels .== dB, :]'
    zA, zB = M'*xA, M'*xB
    
    scatter(zA[1,:], zA[2,:], c=:red, label="Digit $(dA)"; args...)
    scatter!(zB[1,:], zB[2,:], c=:blue, label="Digit $(dB)"; args...)
end

plots = []
for k in 1:5
    push!(plots,compareDigits(2k-2,2k-1))
end
plot(plots...,size = (800, 500), margin = 5mm)

#### A linear classifier

In [ ]:
using Flux: onehotbatch

A = [ones(n_train) X]
Adag = pinv(A)  
tfPM(x) = x ? +1 : -1
yDat(k) = tfPM.(onehotbatch(labels,0:9)'[:,k+1])
bets = [Adag*yDat(k) for k in 0:9]

linear_classify(square_image) = findmax([([1 ; vec(square_image)])'*bets[k] for k in 1:10])[2]-1

In [ ]:
predictions = [linear_classify(test_imgs[:,:,k]) for k in 1:n_test]
confusionMatrix = [sum((predictions .== i) .& (test_labels .== j)) for i in 0:9, j in 0:9]
acc = sum(diag(confusionMatrix))/n_test

println("Accuracy: ", acc, "\nConfusion Matrix:")
show(stdout, "text/plain", confusionMatrix)

#### Training neural networks

In [ ]:
train_range, validate_range = 1:5000, 5001:10000
batch_size = 1000

train_imgs = imgs[:,:,train_range]
train_labels = labels[train_range]
mb_idxs = Iterators.partition(1:length(train_range), batch_size)

validate_imgs = imgs[:,:,validate_range]
validate_labels = labels[validate_range];

In [ ]:
function minibatch(x, y, index_range)
#     xBatch = Array{Float32}(undef, size(x[1])..., 1, length(indexRange))
    x_batch = Array{Float32}(undef, 28,28, 1, length(index_range))
    for i in 1:length(index_range)
        x_batch[:, :, :, i] = Float32.(x[:,:,index_range[i]])
    end
    return (x_batch, onehotbatch(y[index_range], 0:9))
end

train_set = [minibatch(train_imgs, train_labels, bi) for bi in mb_idxs];
@show length(train_set)
validate_set = [minibatch(validate_imgs,validate_labels, 1:length(validate_range))]
@show length(validate_set);

In [ ]:
using Flux: onehotbatch, onecold, crossentropy, flatten, params

In [ ]:
#A dense model
model1 = Chain(flatten, 
                Dense(784, 200, relu),
                Dense(200, 100, tanh),
                Dense(100, 10, sigmoid),
                softmax)

In [ ]:
#A convolutional model
model2 = Chain(
            Conv((5, 5), 1=>8, relu), MaxPool((2,2)),
            Conv((3, 3), 8=>16, relu), MaxPool((2,2)),
            flatten, 
            Dense(400, 10), 
            softmax)

In [ ]:
fix_seed!()

epochs = 30

η = 5e-3 #learning rate
opt1, opt2 = ADAM(η), ADAM(η)

accuracyPaths = [[],[]]

accuracy(x, y, model) = mean(onecold(model(x)) .== onecold(y))
loss(x, y, model) = crossentropy(model(x), y)

cbF1() = begin
            acc = accuracy(first(validate_set)..., model1)
            print("$acc, ")
            push!(accuracyPaths[1],acc)
        end

cbF2() = begin
            acc = accuracy(first(validate_set)..., model2)
            print("$acc, ")
            push!(accuracyPaths[2],acc)
        end

model1(train_set[1][1]); model2(train_set[1][1])

for ep in 1:epochs
    print("\nEpoch $ep \n  Model 1 accuracy: ")
    Flux.train!((x,y)->loss(x,y,model1), params(model1), train_set, opt1, cb=cbF1)
    print("  Model 2 accuracy: ")
    Flux.train!((x,y)->loss(x,y,model2), params(model2), train_set, opt2, cb=cbF2)
end

println("\n\nFinal Model1 (Dense) accuracy = ", accuracy(first(validate_set)..., model1))
println("Final Model2 (Convolutional) accuracy = ", accuracy(first(validate_set)..., model2))

plot(accuracyPaths,label = ["Dense" "Convolutional"],
    ylim=(0.7,1.0), xlabel="Batch number", ylabel = "Validation Accuracy")

[home](#home)

## The End 

I hope you enjoyed the tutorial.

Many more of these types of examples + statistical background are at [statisticswithjulia.org](https://statisticswithjulia.org/).